In [1]:
import pandas as pd
import numpy as np
from time import perf_counter
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
filtered_survival_df.head()

(66, 16)


,sample_name,survival_time,age_at_diagnosis,race_american_indian_or_alaska_native,race_asian,race_black_or_african_american,race_native_hawaiian_or_other_pacific_islander,race_not_reported,race_white,ethnicity_hispanic_or_latino,ethnicity_not_hispanic_or_latino,ethnicity_not_reported,figo_chr_figo_stage_1,figo_chr_figo_stage_2,figo_chr_figo_stage_3,figo_chr_figo_stage_4
0,TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975.0,0,0,0,0,0,1,0,0,1,1,0,0,0
1,TCGA-C5-A8YT-01A-11R-A37O-07,633,13253.0,0,0,0,0,0,1,0,1,0,1,0,0,0
2,TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727.0,0,0,0,0,0,1,0,0,1,1,0,0,0
3,TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444.0,0,0,0,0,0,1,0,1,0,1,0,0,0
4,TCGA-DS-A7WF-01A-11R-A352-07,492,15319.0,0,0,0,0,1,0,1,0,0,1,0,0,0


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


,sample_name,PGF,TIMP4,C1QTNF6,TNC,PRL,OGN,C1QL3,FGB,NDNF,...,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
0,TCGA-C5-A1BF-01B-11R-A13Y-07,10.800637,6.228003,11.669331,13.002928,5.063964,4.869744,5.063964,8.834522,6.410767,...,9.013453,8.190325,9.503647,14.077995,6.569726,7.315604,4.602649,12.062300,5.649441,16.558407
1,TCGA-C5-A8YT-01A-11R-A37O-07,7.830611,5.733875,12.445548,13.765468,5.455125,13.049104,5.146455,5.074289,10.569544,...,9.453187,6.398956,12.288955,13.396332,10.228758,8.542025,4.602649,11.765396,5.318924,13.556322
2,TCGA-C5-A1BE-01B-11R-A13Y-07,10.642039,5.348449,8.945220,13.419225,4.602649,5.867905,5.646251,4.602649,5.673617,...,11.696884,6.384980,9.774029,15.381944,6.205261,7.163053,4.957257,10.113788,5.213815,15.564969
3,TCGA-C5-A8XH-01A-11R-A37O-07,9.633752,5.908552,11.672191,13.863766,4.602649,5.339887,5.702581,5.389600,5.634507,...,11.148165,7.526140,8.561116,14.404419,6.462928,6.102570,4.602649,9.104547,5.088257,15.197350
4,TCGA-DS-A7WF-01A-11R-A352-07,13.292479,5.620704,12.867887,16.646519,4.602649,10.377267,6.177498,4.602649,5.198452,...,10.809104,7.877841,6.615459,13.894278,7.058159,10.809104,4.602649,15.271686,6.519692,16.820793


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
joined_df.head()

(66, 1023)


,survival_time,age_at_diagnosis,race_american_indian_or_alaska_native,race_asian,race_black_or_african_american,race_native_hawaiian_or_other_pacific_islander,race_not_reported,race_white,ethnicity_hispanic_or_latino,ethnicity_not_hispanic_or_latino,...,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
sample_name,,,,,,,,,,,,,,,,,,,,,
TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975.0,0,0,0,0,0,1,0,0,...,9.013453,8.190325,9.503647,14.077995,6.569726,7.315604,4.602649,12.062300,5.649441,16.558407
TCGA-C5-A8YT-01A-11R-A37O-07,633,13253.0,0,0,0,0,0,1,0,1,...,9.453187,6.398956,12.288955,13.396332,10.228758,8.542025,4.602649,11.765396,5.318924,13.556322
TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727.0,0,0,0,0,0,1,0,0,...,11.696884,6.384980,9.774029,15.381944,6.205261,7.163053,4.957257,10.113788,5.213815,15.564969
TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444.0,0,0,0,0,0,1,0,1,...,11.148165,7.526140,8.561116,14.404419,6.462928,6.102570,4.602649,9.104547,5.088257,15.197350
TCGA-DS-A7WF-01A-11R-A352-07,492,15319.0,0,0,0,0,1,0,1,0,...,10.809104,7.877841,6.615459,13.894278,7.058159,10.809104,4.602649,15.271686,6.519692,16.820793


In [10]:
rand.seed(123)
shuffled_df = joined_df.sample(frac=1, random_state=rand)
x_df = shuffled_df.iloc[:, 1:]
y_df = shuffled_df.iloc[:, [0]]

In [11]:
def objective(h_params, X, y):
    print(h_params)
    model = SVR(
        kernel=h_params[0],
        C=h_params[1],
        epsilon=h_params[2],
        gamma=h_params[3],
        degree=h_params[4],
        coef0=h_params[5]
    )
    # Standardize all variables (except one-hots)
    pipeline = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
        ], remainder="passthrough"),
        model
    )
    ttr = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    return -np.mean(cross_val_score(
        ttr,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))


In [24]:
space = [
    Categorical(["linear", "rbf", "poly"], name="kernel"),
    Real(1e-1, 1e1, name="C"),
    Real(1e-3, 1e-1, name="epsilon"),
    Real(1e-3, 1e-1, name="gamma"),
    Integer(1, 3, name="degree"),
    Real(-1, 1, name="coef0")
]
n_initial = 40 * len(space[0].categories)
n_calls = 200 * len(space[0].categories)

In [13]:
res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1
)

Iteration No: 1 started. Evaluating function at random point.
['rbf', 4.860832066690728, 0.07997511248069224, 0.028987590498937163, 2, -0.9804853006583336]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.8231
Function value obtained: 565.4600
Current minimum: 565.4600
Iteration No: 2 started. Evaluating function at random point.
['poly', 6.923792863971573, 0.08706755969427699, 0.0275125783572481, 2, -0.4634373845765508]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.4857
Function value obtained: 648.8427
Current minimum: 565.4600
Iteration No: 3 started. Evaluating function at random point.
['rbf', 2.778095179642898, 0.05347613910877633, 0.018361893417211655, 2, 0.7822181916783213]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.5093
Function value obtained: 558.7666
Current minimum: 558.7666
Iteration No: 4 started. Evaluating function at random point.
['linear', 4.9937304017493425, 0.022017512235608248, 0.052566810001

In [39]:
res.fun

494.27499939111266

In [47]:
h_param_df = pd.DataFrame({
    "param": res.space.dimension_names + ["loss_achieved"],
    "param_value": res.x + [res.fun]
})
h_param_df

,param,param_value
0,kernel,rbf
1,C,0.699564
2,epsilon,0.0133707
3,gamma,0.00101448
4,degree,3
5,coef0,-0.328049
6,loss_achieved,494.275


In [48]:
h_param_df.to_csv("opt_svr_h_params.tsv", sep="\t", index=False)